In [5]:
from numpy import *
from math import *
from pykep import epoch, lambert_problem, fb_vel, DAY2SEC, AU, MU_SUN, RAD2DEG
from pykep.planet import keplerian, jpl_lp, spice

class flyby:

    jpl_planet = {  'mercury'   : jpl_lp('mercury'),
                    'venus'     : jpl_lp('venus'),
                    'earth'     : jpl_lp('earth'),
                    'mars'      : jpl_lp('mars'),
                    'jupiter'   : jpl_lp('jupiter'),
                    'saturn'    : jpl_lp('saturn'),
                    'uranus'    : jpl_lp('uranus'),
                    'neptune'   : jpl_lp('neptune'),
                    'pluto'     : jpl_lp('pluto')
                 }


    mu_sun_spice = 132712440017.99e9
    spice_planet = {'mercury'   : spice('MERCURY', 'SUN', 'ECLIPJ2000', 'NONE', mu_sun_spice, 22032.080486418e9, 2439.7e3, 2439.7e3 * 1.1),
                    'venus'     : spice('VENUS', 'SUN', 'ECLIPJ2000', 'NONE', mu_sun_spice, 324858.59882646e9, 6051.9e3, 6051.9e3 * 1.1),
                    'earth'     : spice('EARTH', 'SUN', 'ECLIPJ2000', 'NONE', mu_sun_spice, 398600.4415e9, 6378.1363e3, 6378.1363e3 * 1.1),
                    'mars'      : spice('MARS', 'SUN', 'ECLIPJ2000', 'NONE', mu_sun_spice, 42828.314258067e9, 3397.0e3, 3397.0e3 * 1.1),
                    'jupiter'   : spice('JUPITER', 'SUN', 'ECLIPJ2000', 'NONE', mu_sun_spice, 126712767.8578e9, 71492.0e3, 71492.0e3 * 1.1),
                    'saturn'    : spice('SATURN', 'SUN', 'ECLIPJ2000', 'NONE', mu_sun_spice, 37940626.061137e9, 60268.0e3, 60268.0e3 * 1.1),
                    'uranus'    : spice('URANUS', 'SUN', 'ECLIPJ2000', 'NONE', mu_sun_spice, 5794549.0070719e9, 25559.0e3, 25559.0e3 * 1.1),
                    'neptune'   : spice('NEPTUNE', 'SUN', 'ECLIPJ2000', 'NONE', mu_sun_spice, 6836534.0638793e9, 25269.0e3, 25269.0e3 * 1.1),
                    'pluto'     : spice('PLUTO', 'SUN', 'ECLIPJ2000', 'NONE', mu_sun_spice, 981.600887707e9, 1162.0e3, 1162.0e3 * 1.1)
        }


    def __init__(self, flight_plan, travel_days, windows, ignore_last=False, orbit_alt=300000, days=1, spice=False, multi_revs=5):

        self.travel_days = travel_days.copy()
        self.windows = windows.copy()
        self.orbit_alt = orbit_alt
        self.ignore_last = ignore_last
        self.dim = len(flight_plan)
        self.flight_plan = flight_plan.copy()
        self.days = days
        self.spice = spice
        self.multi_revs = multi_revs

        self.planets = []
        if self.spice :
            self.mu_sun = self.mu_sun_spice
            for i in flight_plan:
                self.planets.append(self.spice_planet[i])
        else:
            self.mu_sun = MU_SUN
            for i in flight_plan:
                self.planets.append(self.jpl_planet[i])


        self.x = [0] * self.dim
        self.t = [0] * self.dim
        self.r = [0] * self.dim
        self.v = [0] * self.dim
        self.vo = [0] * self.dim
        self.vi = [0] * self.dim
        self.f = [0] * self.dim
        self.f_all = 0
        self.li_sol = []
        self.l = []

    def get_bounds(self):
        return ([-1 * x for x in self.windows], self.windows)
    
    def get_name(self):
        return "Flyby"

    def fitness(self, x):
        self.x = x
        # Расчет времени
        self.t[0] = self.travel_days[0] + self.x[0]
        for i in range(1, self.dim):
            self.t[i] = self.days * self.t[i - 1] + self.travel_days[i] + self.x[i]

        # вычисление вектора состояния планет
        for i in range(self.dim):
            self.r[i], self.v[i] = self.planets[i].eph(epoch(self.t[i], "mjd"))

        # Ламберт 
        self.l = []
        n_sols = []
        for i in range(self.dim - 1):
            self.l.append(lambert_problem(self.r[i], self.r[i + 1], (self.t[i + 1] - self.t[i]) * DAY2SEC, self.mu_sun, False, self.multi_revs))
            n_sols.append(self.l[i].get_Nmax() * 2 + 1)
            
        # расчет dV 
        mu0 = self.planets[0].mu_self
        rad0 = self.planets[0].radius + self.orbit_alt
        mu1 = self.planets[-1].mu_self
        rad1 = self.planets[-1].radius + self.orbit_alt
        
        k = 1
        for i in range(self.dim - 1):
            k = k * n_sols[i]
        
        vot = [0] * self.dim
        vit = [0] * self.dim
        ft = [0] * self.dim
        self.f_all = 1.0e10
        
        for kk in range(k):
            d = kk
            li = []
            for j in range(self.dim - 1):
                d , b = divmod(d , n_sols[j])
                li.append(b)
                
            vot[0] = array(self.l[0].get_v1()[li[0]]) - self.v[0]
            ft[0] = sqrt(dot(vot[0], vot[0]) + 2 * mu0 / rad0) - sqrt(1 * mu0 / rad0)

            if ft[0] > self.f_all:
                continue

            for i in range(1,self.dim - 1):
                vit[i] = array(self.l[i - 1].get_v2()[li[i - 1]]) - self.v[i]
                vot[i] = array(self.l[i].get_v1()[li[i]]) - self.v[i]
                ft[i] = fb_vel(vit[i], vot[i], self.planets[i])
            
            vit[-1] = array(self.l[-1].get_v2()[li[-1]]) - self.v[-1]
            ft[-1] = sqrt(dot(vit[-1], vit[-1]) + 2 * mu1 / rad1) - sqrt(2 * mu1 / rad1)
            
            ft_all = sum(ft)
            if self.ignore_last :
                ft_all = ft_all - ft[-1]

            if ft_all < self.f_all:
                self.f_all = ft_all
                self.vi = vit.copy()
                self.vo = vot.copy()
                self.f = ft.copy()
                self.li_sol = li.copy()


        # проверка отрицательной высоты пролета
        res = self.f_all
        for i in range(1, self.dim - 1):
            ta = acos(dot(self.vi[i], self.vo[i]) / sqrt(dot(self.vi[i], self.vi[i])) / sqrt(dot(self.vo[i], self.vo[i])))
            alt = (self.planets[i].mu_self / dot(self.vi[i], self.vi[i]) * (1 / sin(ta / 2) - 1)) - self.planets[i].safe_radius
            if alt < 0:
                res = res - alt

        # вернуть общую стоимость топлива
        return [res]

    def get_extra_info(self):
        return "\tDimensions: " + str(self.dim)

    def plot_trajectory(self):
        from matplotlib.pyplot import figure, show
        from pykep.orbit_plots import plot_planet, plot_lambert

        fig = figure()
        ax = fig.gca(projection='3d', aspect='equal')
        ax.scatter([0], [0], [0], color='y')
 

        colors = {'mercury': '#7B7869', 
                  'venus'  : '#BB91A1',
                  'earth'  : '#0000FF',
                  'mars'   : '#E27B58',
                  'jupiter': '#C88B3A',
                  'saturn' : '#A49B72',
                  'uranus' : '#65868B',
                  'neptune': '#6081FF',
                  'pluto'  : '#333333'}

        d_max = 0
        for i in range(self.dim):
            r, v = self.planets[i].eph(epoch(self.t[i], "mjd"))
            d = dot(r,r)
            if d > d_max:
                d_max = d

            p = keplerian(epoch(self.t[i], "mjd"),
                          self.planets[i].osculating_elements(epoch(self.t[i], "mjd")),
                          self.planets[i].mu_central_body,
                          self.planets[i].mu_self,
                          self.planets[i].radius,
                          self.planets[i].safe_radius,
                          self.flight_plan[i])
            plot_planet(p, epoch(self.t[i], "mjd"),  units=AU, ax=ax, color=colors[self.flight_plan[i]])
            if i != self.dim - 1:
                plot_lambert(self.l[i], sol=self.li_sol[i], units=AU, ax=ax, color='c')

    
        d_max = 1.2 * sqrt(d_max) / AU
        ax.set_xlim(-d_max, d_max)
        ax.set_ylim(-d_max, d_max)
        ax.set_zlim(-d_max, d_max)

        show()

    def print_transx(self):

        print("Date of %8s departue : " % self.flight_plan[0], epoch(self.t[0], "mjd"))
        for i in range(1,self.dim - 1):
            print("Date of %8s encounter: " % self.flight_plan[i], epoch(self.t[i], "mjd"))
        print("Date of %8s arrival  : " % self.flight_plan[-1], epoch(self.t[-1], "mjd"))
        print("")

        for i in range(self.dim - 1):
            print("Transfer time from %8s to %8s:" % (self.flight_plan[i], self.flight_plan[i + 1]), round(self.t[i + 1] - self.t[i], 4), " days")
        
        print("Total mission duration:                 ", round(self.t[-1] - self.t[0], 4), " days")
        print("")
        print("")

        fward = [0] * self.dim
        plane = [0] * self.dim
        oward = [0] * self.dim
        for i in range(self.dim):
            fward[i] = self.v[i] / linalg.norm(self.v[i])
            plane[i] = cross(self.v[i], self.r[i])
            plane[i] = plane[i] / linalg.norm(plane[i])
            oward[i] = cross(plane[i], fward[i])

        print("TransX escape plan -  %8s escape" % self.flight_plan[0])
        print("--------------------------------------")
        print("MJD:                  %10.4f " % round(epoch(self.t[0], "mjd").mjd, 4))
        print("Prograde:             %10.4f m/s" % round(dot(fward[0], self.vo[0]), 4))
        print("Outward:              %10.4f m/s" % round(dot(oward[0], self.vo[0]), 4))
        print("Plane:                %10.4f m/s" % round(dot(plane[0], self.vo[0]), 4))
        print("Hyp. excess velocity: %10.4f m/s" % round(linalg.norm(self.vo[0]), 4))
        print("Earth escape burn:    %10.4f m/s" % round(self.f[0], 4))

        c3 = dot(self.vo[0],self.vo[0]) / 1000000
        dha = atan2(self.vo[0][2],sqrt(self.vo[0][0] * self.vo[0][0] + self.vo[0][1] * self.vo[0][1])) * RAD2DEG
        rha = atan2(self.vo[0][1],self.vo[0][0]) * RAD2DEG

        print("GMAT MJD:             ", epoch(self.t[0], "mjd").mjd - 29999.5)
        print("GMAT OutgoingC3:      ", c3)
        print("GMAT OutgoingRHA:     ", rha)
        print("GMAT OutgoingDHA:     ", dha)
        print("")

        for i in range(1, self.dim - 1):
            vx = dot(fward[i], self.vo[i])
            vy = dot(oward[i], self.vo[i])
            vz = dot(plane[i], self.vo[i])
            mu = self.planets[i].mu_self
            rad = self.planets[i].radius
            
            print("Vx:     ", vx)##############################
            print("Vy:     ", vy)##############################
            print("Vz:     ", vz)##############################
            
            print("%8s encounter" % self.flight_plan[i])
            print("--------------------------------------")
            print("MJD:                 %10.4f " % round(epoch(self.t[i], "mjd").mjd, 4))
            print("Solution number:     %10d " % (1 + self.li_sol[i - 1]))
            print("Approach velocity:   %10.4f m/s" % round(linalg.norm(self.vi[i]), 4))
            print("Departure velocity:  %10.4f m/s" % round(linalg.norm(self.vo[i]), 4))
            print("Outward angle:       %10.4f deg" % round(atan2(vy, vx) * RAD2DEG, 4))
            print("Inclination:         %10.4f deg" % round(atan2(vz, sqrt(vx * vx + vy * vy)) * RAD2DEG, 4))

            a = - mu / dot(self.vi[i], self.vi[i])
            ta = acos(dot(self.vi[i], self.vo[i]) / linalg.norm(self.vi[i]) / linalg.norm(self.vo[i]))
            e = 1 / sin(ta / 2)
            rp = a * (1 - e) 
            alt = (rp - rad) / 1000
           
            print("Turning angle:       %10.4f deg" % round(ta * RAD2DEG, 4)) 
            print("Periapsis altitude:  %10.4f km " % round(alt, 4))
            print("dV needed:           %10.4f m/s" % round(self.f[i], 4))
            print("GMAT MJD:             ", epoch(self.t[i], "mjd").mjd - 29999.5)
            print("GMAT RadPer:          ", rp / 1000)

            c3 = dot(self.vi[i],self.vi[i]) / 1000000
            dha = atan2(-self.vi[i][2],sqrt(self.vi[i][0] * self.vi[i][0] + self.vi[i][1] * self.vi[i][1])) * RAD2DEG
            rha = atan2(-self.vi[i][1],-self.vi[i][0]) * RAD2DEG
            if rha < 0:
                rha = 360 + rha

            print("GMAT IncomingC3:      ", c3)
            print("GMAT IncomingRHA:     ", rha)
            print("GMAT IncomingDHA:     ", dha)

            e = cross([0,0,1],-self.vi[i])
            e = e / linalg.norm(e)
            n = cross(-self.vi[i],e)
            n = n / linalg.norm(n)
            h = cross(self.vi[i],self.vo[i])
            b = cross(h,-self.vi[i])
            b = b / linalg.norm(b)
            sinb = dot(b,e)
            cosb = dot(b,-n)
            bazi = atan2(sinb,cosb) * RAD2DEG
            if bazi < 0:
                bazi = bazi + 360
            print("GMAT IncomingBVAZI:   ", bazi) 

            c3 = dot(self.vo[i],self.vo[i]) / 1000000
            dha = atan2(self.vo[i][2],sqrt(self.vo[i][0] * self.vo[i][0] + self.vo[i][1] * self.vo[i][1])) * RAD2DEG
            rha = atan2(self.vo[i][1],self.vo[i][0]) * RAD2DEG
            if rha < 0:
                rha = 360 + rha
           
            print("GMAT OutgoingC3:      ", c3)
            print("GMAT OutgoingRHA:     ", rha)
            print("GMAT OutgoingDHA:     ", dha)


            e = cross([0,0,1],self.vo[i])
            e = e / linalg.norm(e)
            n = cross(self.vo[i],e)
            n = n / linalg.norm(n)
            h = cross(self.vi[i],self.vo[i])
            b = cross(h,self.vo[i])
            b = b / linalg.norm(b)
            sinb = dot(b,e)
            cosb = dot(b,-n)
            bazi = atan2(sinb,cosb) * RAD2DEG
            if bazi < 0:
                bazi = bazi + 360
            print("GMAT OutgoingBVAZI:   ", bazi) 

            print("")


        print("%8s arrival" % self.flight_plan[-1])
        print("--------------------------------------")
        print("MJD:                  %10.4f    " % round(epoch(self.t[-1], "mjd").mjd, 4))
        print("Solution number:      %10d " % (1 + self.li_sol[-1] + 1))
        print("Hyp. excess velocity: %10.4f m/s" % round(sqrt(dot(self.vi[-1], self.vi[-1])), 4))
        print("Orbit insertion burn  %10.4f m/s - C3 = 0" % round(self.f[-1], 4))

        c3 = dot(self.vi[-1],self.vi[-1]) / 1000000
        dha = atan2(self.vi[-1][2],sqrt(self.vi[-1][0] * self.vi[-1][0] + self.vi[-1][1] * self.vi[-1][1])) * RAD2DEG
        rha = atan2(self.vi[-1][1],self.vi[-1][0]) * RAD2DEG

        print("GMAT MJD:             ", epoch(self.t[-1], "mjd").mjd - 29999.5)
        print("GMAT IncomingC3:      ", c3)
        print("GMAT IncomingRHA:     ", rha)
        print("GMAT IncomingDHA:     ", dha)
        print("")


        print("--------------------------------------")
        print("Total fuel cost:     %10.4f m/s" % round(self.f_all, 4))

In [6]:
    from pygmo import archipelago, problem, sade
    
    p = flyby(['earth','venus', 'earth', 'mars', 'earth', 'jupiter'],
              [60094.0, 60240.0, 60555.0, 60717.0, 61370.0, 62432.0], 
              [15, 15, 15, 15, 15, 15], ignore_last=False, days=0)

    archi = archipelago(n=4, algo=sade(gen = 100), prob=problem(p), pop_size=10)
    archi.evolve(100)
    archi.wait()

    tmp = [isl for isl in archi]
    tmp.sort(key=lambda x: x.get_population().champion_f)

    p.fitness(tmp[0].get_population().champion_x)
    p.print_transx()   

Date of    earth departue :  2023-May-24 23:15:33.296789
Date of    venus encounter:  2023-Oct-22 15:32:12.585035
Date of    earth encounter:  2024-Sep-04 13:03:11.292382
Date of     mars encounter:  2025-Feb-09 23:23:03.000458
Date of    earth encounter:  2026-Nov-23 02:20:53.763018
Date of  jupiter arrival  :  2029-Oct-12 18:05:28.081165

Transfer time from    earth to    venus: 150.6782  days
Transfer time from    venus to    earth: 317.8965  days
Transfer time from    earth to     mars: 158.4305  days
Transfer time from     mars to    earth: 651.1235  days
Transfer time from    earth to  jupiter: 1054.656  days
Total mission duration:                  2332.7847  days


TransX escape plan -     earth escape
--------------------------------------
MJD:                  60088.9691 
Prograde:             -2456.5187 m/s
Outward:               -555.2526 m/s
Plane:                 -810.4018 m/s
Hyp. excess velocity:  2645.6645 m/s
Earth escape burn:     3515.9011 m/s
GMAT MJD:             

In [3]:
    from pykep.util import load_spice_kernel
    load_spice_kernel('D:\Program Files\GMAT\data\planetary_ephem\spk\DE424AllPlanets.bsp')

    from pygmo import archipelago, problem, sade
    p = flyby(['earth','venus','earth','earth', 'jupiter'],
              [62445, 146, 318, 827, 754], 
              [1, 10, 20, 30, 40 ] ,  days = 1, spice = True, multi_revs=3)

    archi = archipelago(n=4, algo=sade(gen = 1000), prob=problem(p), pop_size=30)
    archi.evolve(100)
    archi.wait()

    tmp = [isl for isl in archi]
    tmp.sort(key=lambda x: x.get_population().champion_f)

    p.fitness(tmp[0].get_population().champion_x)
    p.print_transx()


Date of    earth departue :  2029-Nov-05 03:14:30.263705
Date of    venus encounter:  2030-Mar-30 21:30:16.337551
Date of    earth encounter:  2031-Feb-15 02:00:49.873369
Date of    earth encounter:  2033-May-23 21:09:10.660667
Date of  jupiter arrival  :  2035-May-29 23:02:18.994398

Transfer time from    earth to    venus: 145.7609  days
Transfer time from    venus to    earth: 321.1879  days
Transfer time from    earth to    earth: 828.7975  days
Transfer time from    earth to  jupiter: 736.0786  days
Total mission duration:                  2031.8249  days


TransX escape plan -     earth escape
--------------------------------------
MJD:                  62445.1351 
Prograde:             -2733.5513 m/s
Outward:              -1100.7130 m/s
Plane:                 1451.9291 m/s
Hyp. excess velocity:  3285.1134 m/s
Earth escape burn:     3683.3026 m/s
GMAT MJD:              32445.63507249659
GMAT OutgoingC3:       10.791969944257302
GMAT OutgoingRHA:      -68.54054000507544
GMAT Outgo